In [1]:
from pathlib import Path
import sys
GIT_ROOT = Path("../..").resolve()
print(GIT_ROOT)
SRC = GIT_ROOT / "src"
if not str(SRC) in sys.path:
    sys.path.append(str((SRC)))
import torch


from utils.tools import DEVICE, DTYPE
from utils.data import get_htc2022_train_phantoms, get_htclike_train_phantoms
from utils.polynomials import Legendre, Chebyshev
from geometries import HTC2022_GEOMETRY

from models.modelbase import plot_model_progress, evaluate_batches
from models.FNOBPs.mifno_bp import IterativePjectingFNO


/home/emastr/deep-limited-angle/KEX---CT-reconstruction


In [4]:
from statistics import mean

PHANTOMS = torch.concat([get_htc2022_train_phantoms(), get_htc2022_train_phantoms()])[:2]
geometry = HTC2022_GEOMETRY
ar = 0.25
SINOS = geometry.project_forward(PHANTOMS)

mifno_iter = IterativePjectingFNO(geometry, 5, 15, [[40,40]]*5, Legendre.key)
dataset = torch.utils.data.TensorDataset(PHANTOMS, SINOS)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.Adam(mifno_iter.parameters(), lr=1e-3, weight_decay=1e-6)
n_epochs = 100

for epoch in range(n_epochs):

    batch_losses, batch_recon_losses = [], []

    for phantom_batch, sino_batch in dataloader:
        optimizer.zero_grad()

        la_sinos, known_angles = geometry.zero_cropp_sinos(sino_batch, ar, 0)
        la_sios, known_region = geometry.reflect_fill_sinos(la_sinos, known_angles)
        known_angles = (~known_region).sum(dim=-1) == 0
        exp_sinos = la_sinos + 0
        loss = torch.tensor(0, device=DEVICE, dtype=DTYPE)
        for fno in mifno_iter.fnos:
            exp_sinos[:, ~known_region] = torch.nn.functional.relu(geometry.project_sinos(fno(exp_sinos), Legendre, mifno_iter.n_degs))[:, ~known_region]
            loss += torch.mean((exp_sinos-sino_batch)**2) / mifno_iter.n_iters
        
        recons = torch.nn.functional.relu(geometry.fbp_reconstruct(exp_sinos))
        recon_loss = torch.mean((recons-phantom_batch)**2)
        loss += recon_loss
        loss.backward()
        optimizer.step()

        batch_losses.append(loss.item())
        batch_recon_losses.append(recon_loss.item())

    print("Epoch", epoch+1, "loss:", mean(batch_losses), "recon loss:", mean(batch_recon_losses))

        


Epoch 1 loss: 1303.0601806640625 recon loss: 0.1995367629826223
Epoch 2 loss: nan recon loss: nan
Epoch 3 loss: nan recon loss: nan


KeyboardInterrupt: 

In [5]:
la_sinos, known_angles = geometry.zero_cropp_sinos(SINOS, ar, 0)
import matplotlib
matplotlib.use("WebAgg")

plot_model_progress(mifno_iter, SINOS[:10], known_angles, known_angles|1, 0)

RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [7]:
from models.modelbase import save_model_checkpoint

save_model_checkpoint(mifno_iter, optimizer, loss, ar, GIT_ROOT / "data/models/mifno_iter_ar_0.25_draft1.0.pt")

RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.